In [1]:
import numpy as np
from astropy.io import ascii
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
print 'Reading files'
w = ascii.read('wen.csv') 
r = fits.open('redmapper.fits') 
r = r[1].data
print len(w)
print len(r)

Reading files
132684
26111


In [11]:
# RA e DEC - catalogo Wen
#w['RAJ2000'], w['DEJ2000']
# RA e DEC catalogo Redmapper
#r['RA'], r['DEC']

RA = w['RAJ2000']
DEC = w['DEJ2000']
ra = r['RA']
dec = r['DEC']

In [10]:
def mask_from_footprint(VISIB_MASK,RA,DEC,DIR='',CTYP=''):
    import pyfits as pf
    import healpy as hpx

    printm(DIR,"######################################")
    printm(DIR,'## READING %s footprint ## '%CTYP)
    printm(DIR,VISIB_MASK)
    hdu = pyfits.open(VISIB_MASK)
    data = hdu[1].data

    #ra_m = data.field('ra')
    #dec_m = data.field('dec')
    pixel_m = data.field('pixel')
    #signal_m = data.field('signal')

    NSIDE = 4096

    phi   = deg_to_rad*np.array(RA)        # phi   = RA.       Azimuthal Angle.
    theta = deg_to_rad*np.array(90.0-DEC)  # theta = pi/4-DEC. Polar Angle.
    pixel = hpx.pixelfunc.ang2pix(NSIDE, theta, phi, nest=False)

    pix_dic = dict(np.vstack([pixel_m,pixel_m]).T)

    printm(DIR,"## creating visibility mask ##")
    mask_aux = np.zeros(len(RA))
    for i,p in enumerate(pixel):
        if p in pix_dic:
            mask_aux[i] = 1

    return (mask_aux == 1)

In [17]:
def get_nside(ra,dec,val):
    import healpy as hpx

    phi   = deg_to_rad*np.array(ra)        # phi   = RA.       Azimuthal Angle.
    theta = deg_to_rad*np.array(90.0-dec)  # theta = pi/4-DEC. Polar Angle.

    pix_dens = 0

    for n in range(1,12):
        NSIDE_USE = 2**n
        pixel = hpx.pixelfunc.ang2pix(NSIDE_USE, theta, phi, nest=False)

        hist = np.histogram(pixel,bins=range(12*NSIDE_USE**2))[0]
        pix_dens = np.mean(hist[hist>0])

        if pix_dens < val:
            break

    return NSIDE_USE, pixel

In [4]:
def create_footprint(ra, dec, hdu, FNAME, CTYP, DIR='', NSIDE=4096):

    prt_title(DIR,'CREATING %s footprint'%CTYP)

    import pyfits as pf
    import healpy as hpx

    # actually creates a mask with pixel density < 2

    NSIDE_USE, pixel = get_nside(ra, dec, 2)

    print CTYP,'FOOTPRINT NSIDE =',NSIDE_USE

    # change mask to NSIDE

    map0 = np.zeros(12*NSIDE_USE**2)
    for p in set(pixel): map0[p] = 1.

    map = hpx.ud_grade(map0, NSIDE)
    pixel = np.arange(12*NSIDE**2)[ map==1. ]

    # print mask

    match_label = [ "pixel" , "ra"    , "dec"    , "signal"]
    col_format  = [ "J", "E", "E", "E"]
    columns = [pixel]

    col_prt = [pyfits.Column(name=n,format=f,array=a) for n,f,a in zip(match_label,col_format,columns)]
    cols = pyfits.ColDefs(col_prt)

    printm (col_prt)
    printm ("cols:\n",cols)
    if hasattr(pyfits.BinTableHDU, 'from_columns'):
        tbhdu = pyfits.BinTableHDU.from_columns(cols)
    else:
        tbhdu = pyfits.new_table(cols)

    thdulist = pyfits.HDUList([hdu[0],tbhdu])
    os.system("rm -f "+str(FNAME))
    thdulist.writeto(FNAME)

    prt_title(DIR,'CREATING %s footprint'%CTYP,end=True)

    return NSIDE_USE

In [5]:
def create_all_footprints(config):

    cluster = set_cat_obj(config)
    halo = set_cat_obj(config,cluster=False)

    mlims = {'r':config.RICHMIN, 'm':config.MASSMIN, 'l':config.LUMMIN}

    cluster.crop_mask = np.array(crop_mask0(config,cluster.rich,cluster.ra,cluster.dec,cluster.z,mlims[cluster.mtype]))
    halo.crop_mask = np.array(crop_mask0(config,halo.mass,halo.ra,halo.dec,halo.z,mlims[halo.mtype]))

    cluster.mask_obj(cluster.crop_mask)
    halo.mask_obj(halo.crop_mask)

    FPRT_NAME = '%s_artificial_footprint.fits'
    NSIDE = 4096

    NSIDE_cl, NSIDE_h = '-', '-'

    if config.CL_MASK is None:
        config.CL_MASK = FPRT_NAME%'cluster'
        NSIDE_cl = create_footprint(cluster.ra, cluster.dec, cluster.hdu, config.CL_MASK, 'CLUSTER', DIR=config.DIR, NSIDE=NSIDE)

    if config.H_MASK is None:
        config.H_MASK = FPRT_NAME%'halo'
        NSIDE_h = create_footprint(halo.ra, halo.dec, halo.hdu, config.H_MASK, 'HALO', DIR=config.DIR, NSIDE=NSIDE)

    f=open('../footprints_NSIDE.txt','w')
    printm('Cluster :',NSIDE_cl,file=f)
    printm('Halo :',NSIDE_h,file=f)
    f.close()